<a href="https://colab.research.google.com/github/El-Eng/freecodecamp/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [165]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [166]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

#!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [167]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [168]:
# add your code here - consider creating a new cell for each section of code

Remove ratings for ISBN's that arent in the books database

In [169]:
#df_r = df_ratings[df_ratings['isbn'].isin(df_books['isbn'])]
df_r = df_ratings

Check duplicate books

In [170]:
df_books[['title', 'author']].value_counts()

title                                                     author           
Little Women                                              Louisa May Alcott    21
Wuthering Heights                                         Emily Bronte         20
Adventures of Huckleberry Finn                            Mark Twain           20
Pride and Prejudice                                       Jane Austen          18
Jane Eyre                                                 Charlotte Bronte     16
                                                                               ..
Heritage of Shame                                         Meg Hutchinson        1
Heritage of Stars                                         Clifford D. Simak     1
Heritage of Western Civilization, Vol. 2, Eighth Edition  John L. Beatty        1
Heritage: Civilization and the Jews                       Abba Solomon Eban     1
Ã?Â?thique en toc                                         Didier Daeninckx      1
Length: 251203, dtype:

In [171]:
df_books[(df_books['title'] == 'Little Women') & (df_books['author'] == 'Louisa May Alcott') ]

,isbn,title,author
4330,0861782356,Little Women,Louisa May Alcott
12228,0590203509,Little Women,Louisa May Alcott
20677,0440447682,Little Women,Louisa May Alcott
32838,1550134140,Little Women,Louisa May Alcott
52213,0451529308,Little Women,Louisa May Alcott
52554,0451518144,Little Women,Louisa May Alcott
53361,0590437976,Little Women,Louisa May Alcott
72171,0785788727,Little Women,Louisa May Alcott
82568,0448110199,Little Women,Louisa May Alcott
84810,0812523334,Little Women,Louisa May Alcott


**Can add code here to remove instances of multiple isbn numbers for the same book, and add an addional "unique book id" column to the df_books based of book, author pairs, then add the same column to the df_ratings. Then use this as the index for the transformation and suggestion algorithm.**

duplicate_isbn = df_books.groupby(['title', 'author']).first().reset_index()

Remove authors and books with less than 20 reviews

In [172]:
book_rev_num = 100
user_rev_num = 200

In [173]:
df_r_rmu = df_r[ df_r['user'].isin((df_r['user'].value_counts() > user_rev_num)[df_r['user'].value_counts() > user_rev_num].index)]

In [184]:
df_r_rmb = df_r_rmu[ df_r['isbn'].isin((df_r['isbn'].value_counts() > book_rev_num)[df_r['isbn'].value_counts() > book_rev_num].index)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


dataframe too big to .pivot, so do it in chunks

In [186]:
df_ratings_pivot = df_r_rmb.pivot(
    index='isbn',
    columns='user',
    values='rating'
).fillna(0)

df_ratings_pivot.index = df_ratings_pivot.join(df_books.set_index('isbn'))['title']

In [187]:
# convert dataframe of movie features to scipy sparse matrix
mat_ratings = csr_matrix(df_ratings_pivot.values)

In [177]:
# # https://stackoverflow.com/questions/61757170/python-unstacked-dataframe-is-too-big-causing-int32-overflow
# from tqdm import tqdm
# chunk_size = 100
# chunks = [x for x in range(0, df_ratings_clean.shape[0], chunk_size)]

# df_ratings_pivot = pd.DataFrame()

# for i in tqdm(range(0, len(chunks) - 1)):
#     chunk_df = df_ratings_clean.iloc[ chunks[i]:chunks[i + 1] - 1]
#     interactions = chunk_df.pivot(
#       index='isbn',
#       columns='user',
#       values='rating'
#     )
#     df_ratings_pivot = df_ratings_pivot.append(interactions, sort=False) 

In [188]:
model_knn = NearestNeighbors(metric='cosine')#, algorithm='brute', n_neighbors=20, n_jobs=-1)

In [189]:
model_knn.fit(mat_ratings)

NearestNeighbors(metric='cosine')

In [190]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  distance, indice = model_knn.kneighbors([df_ratings_pivot.loc[book].values], n_neighbors=6)
  output = np.column_stack((df_ratings_pivot.iloc[indice[0]].index.values[1:], distance[0][1:].astype(float)))
  recommended_books = [book, output[output[:,1].argsort()[::-1]] ]
  return recommended_books



# [
#   'The Queen of the Damned (Vampire Chronicles (Paperback))',
#   [
#     ['Catch 22', 0.793983519077301], 
#     ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
#     ['Interview with the Vampire', 0.7345068454742432],
#     ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
#     ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
#   ]
# ]


Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [191]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074],
       ['The Lovely Bones: A Novel', 0.7230184078216553]], dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉


In [182]:
book = "The Queen of the Damned (Vampire Chronicles (Paperback))"



# [
#   'The Queen of the Damned (Vampire Chronicles (Paperback))',
#   [
#     ['Catch 22', 0.793983519077301], 
#     ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
#     ['Interview with the Vampire', 0.7345068454742432],
#     ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
#     ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
#   ]
# ]
